In [44]:
from PIL import Image
import numpy as np
from src.localreader import LocalReader, SHRINK, CENTERING, getMean

In [7]:
from keras.models import Model, load_model
from keras.layers import Input, Conv2D, MaxPooling2D, Activation, Flatten, Dense, BatchNormalization
from keras.optimizers import Adam

In [2]:
model = load_model("full-weights-improvement-04-0.65.hdf5")

In [19]:
reader = LocalReader(width=256, height=256, shaping=SHRINK, validation_size=0.0, img_path='data/images/testing')

In [31]:
def test_generator(reader):
    for images, scores in reader.train(end=167):
        for i in range(0, len(images), 32):
            imgs = images[i:i+32]
            scrs = scores[i:i+32]
            yield (imgs, scrs)

In [32]:
res = model.evaluate_generator(test_generator(reader), steps=59)

In [30]:
#test loss (mean squared error)
res

[0.6294767776968245, 0.6294767776968245]

### Get predictions

In [88]:
def get_true_values(reader):
    true_values = []
    imgs = []
    for images, scores in reader.train(end=167):
        true_values.append(scores)
        imgs.append(images)
    return np.concatenate(true_values), np.concatenate(imgs)

def show_img(data):
    img = Image.fromarray(data, 'RGB')
    img.show()

In [37]:
predictions = model.predict_generator(test_generator(reader), steps=59)

In [66]:
true_values, images = get_true_values(reader)

In [64]:
len(predictions) == len(true_values)

True

In [67]:
for i in range(10):
    print("true:", true_values[i], "\t", "predicted:", predictions[i][0])

true: 4.9953271028037385 	 predicted: 5.4986243
true: 4.921568627450981 	 predicted: 5.128674
true: 5.792270531400966 	 predicted: 5.2465067
true: 5.752475247524752 	 predicted: 5.35035
true: 6.228971962616822 	 predicted: 5.7382054
true: 6.6761904761904765 	 predicted: 4.9721026
true: 4.898058252427185 	 predicted: 5.3504405
true: 6.172897196261682 	 predicted: 5.285688
true: 5.966346153846154 	 predicted: 5.437385
true: 3.9121951219512194 	 predicted: 5.7300835


In [90]:
#image with highest and lowest predicted scores
highest = np.argmax(predictions)
lowest = np.argmin(predictions)

In [81]:
print("predicted highest:", predictions[highest][0], "true:", true_values[highest])

predicted highest: 14.229767 true: 5.039823008849558


In [79]:
print("predicted lowest:", predictions[lowest][0], "true:", true_values[lowest])

predicted lowest: 4.3192706 true: 5.321212121212121


In [91]:
show_img(images[highest])
show_img(images[lowest])

In [92]:
#image with highest and lowest true scores
highest = np.argmax(true_values)
lowest = np.argmin(true_values)

In [93]:
print("true highest:", true_values[highest], "predicted:", predictions[highest][0])

true highest: 7.861538461538461 predicted: 5.2966785


In [94]:
print("true lowest:", true_values[lowest], "predicted:", predictions[lowest][0])

true lowest: 3.092233009708738 predicted: 5.006703


In [95]:
show_img(images[highest])
show_img(images[lowest])

In [99]:
#largest difference in predictions
largest = 0
for i in range(len(true_values)):
    if(abs(true_values[i] - predictions[i][0])) > largest:
        largest = i

In [101]:
print("true:", true_values[largest], "predicted:", predictions[largest][0])

true: 4.810650887573964 predicted: 12.18195


In [105]:
show_img(images[largest])